In [1]:
from datetime import datetime
from datetime import timedelta
from pandas import DataFrame, read_csv
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go

file = 'Time_Range_Test_Case_1.xls'
df = pd.read_excel(file)

# for header in ['ETB', 'ETU', 'QC Seq Time From', 'QC Seq Time To']:
#     for i in range(len(df[header])):
#         day = int(df[header][i][:2])
#         month = int(df[header][i][3:5])
#         year = int(df[header][i][6:10])
#         hour = int(df[header][i][11:13])
#         minute = int(df[header][i][14:16])
#         second = int(df[header][i][17:19])
#         df[header][i]= datetime(year, month, day, hour, minute, second, 0)

df = df.drop(columns=['Terminal', 'Voyage', 'Berthing Sequence']);

df['timeBefore'] = df['QC Seq Time From'] - df['ETB']

df['timeAfter'] = df['ETU'] - df['QC Seq Time To'] 

df['Quantity'] = 2

df['Type']='Berthing'

df['Note']=False

In [2]:
JP_LOADING_TIME = timedelta(hours=1)
TRAVEL_TIME = timedelta(hours=1)
MAX_JP_WAIT_TIME = timedelta(hours=1)
PSA_LOADING_TIME= timedelta(hours=2)
WAREHOUSE_LOADING = timedelta(minutes=30)
DRIVER_HOURS= timedelta(hours=12)
MAX_CARGO = 12

In [3]:
df

,Vessel Name,ETB,ETU,QC Seq Time From,QC Seq Time To,timeBefore,timeAfter,Quantity,Type,Note
0,KOTA NAZIM,2019-07-31 13:00:00,2019-08-01 19:00:00,2019-07-31 22:00:00,2019-08-01 02:30:00,09:00:00,0 days 16:30:00,2,Berthing,False
1,KOTA WARIS,2019-07-31 23:00:00,2019-08-02 03:00:00,2019-08-01 00:00:00,2019-08-01 02:30:00,01:00:00,1 days 00:30:00,2,Berthing,False
2,HONGKONG E,2019-07-31 23:00:00,2019-08-02 03:00:00,2019-08-01 00:00:00,2019-08-02 01:30:00,01:00:00,0 days 01:30:00,2,Berthing,False
3,PAC BANDA,2019-08-01 11:30:00,2019-08-02 23:00:00,2019-08-02 00:00:00,2019-08-02 08:30:00,12:30:00,0 days 14:30:00,2,Berthing,False
4,ANTUNG,2019-07-31 10:30:00,2019-07-31 18:00:00,2019-07-31 14:00:00,2019-07-31 15:00:00,03:30:00,0 days 03:00:00,2,Berthing,False
5,LADY OF LUCK,2019-07-31 10:30:00,2019-08-01 09:30:00,2019-07-31 17:00:36,2019-07-31 17:30:00,06:30:36,0 days 16:00:00,2,Berthing,False
6,SEAMAX BRIDG,2019-07-31 14:30:00,2019-08-01 01:30:00,2019-07-31 16:30:00,2019-08-01 00:30:00,02:00:00,0 days 01:00:00,2,Berthing,False
7,LAS VEGAS,2019-07-31 13:30:00,2019-08-01 05:00:00,2019-07-31 14:00:00,2019-08-01 03:00:00,00:30:00,0 days 02:00:00,2,Berthing,False


In [4]:
loadBefore = df[df['timeBefore'].apply(lambda o:o.seconds/60 > 90)]

loadAfter = df[df['timeBefore'].apply(lambda o:o.seconds/60 <= 90)]

#Delivery for Load Before
loadBefore['deliveryTime'] = loadBefore['QC Seq Time From'] - timedelta(hours=2)

loadBefore.sort_values(by=['QC Seq Time From','timeBefore'], inplace=True)

loadBefore = loadBefore.reset_index()

def checkDeliveryTimeBefore():
    updated = False;
    for index, row in loadBefore.iterrows():
        if updated:
            return checkDeliveryTimeBefore()
        else:
            if index == 0:
                continue;
            if (loadBefore['deliveryTime'][index] - PSA_LOADING_TIME) < loadBefore['deliveryTime'][index-1]:
                loadBefore['deliveryTime'][index-1] = loadBefore['deliveryTime'][index] - PSA_LOADING_TIME
                updated = True;
    updated = False
checkDeliveryTimeBefore()

loadBefore['Note'] = loadBefore['deliveryTime'] < loadBefore['ETB']


In [5]:
loadAfter.sort_values(by=['timeAfter'], inplace=True, ascending= True)
loadAfter.sort_values(by=['QC Seq Time To'], inplace=True)
loadAfter = loadAfter.reset_index()

# Do Delivery for Load After
loadAfter['deliveryTime'] = loadAfter['QC Seq Time To']

if loadAfter['deliveryTime'][0] < (loadBefore['deliveryTime'][len(loadBefore['deliveryTime'])-1] + PSA_LOADING_TIME):
    loadAfter['deliveryTime'][0] = loadBefore['deliveryTime'][len(loadBefore['deliveryTime'])-1] + PSA_LOADING_TIME

def checkDeliveryTimeAfter():
    updated = False;
    for index, row in loadAfter.iterrows():
        if updated:
            return checkDeliveryTimeAfter()
        else:
            if index == len(loadAfter)-1:
                continue;
            if (loadAfter['deliveryTime'][index] + PSA_LOADING_TIME) > loadAfter['deliveryTime'][index+1]:
                loadAfter['deliveryTime'][index+1] = loadAfter['deliveryTime'][index] + PSA_LOADING_TIME
                updated = True;
    updated = False
checkDeliveryTimeAfter()

loadAfter['Note'] = (loadAfter['ETU']-loadAfter['deliveryTime']).apply(lambda o:o.seconds/60 < 90) | (loadAfter['ETU']<loadAfter['deliveryTime'])


In [6]:
df_sorted = pd.concat([loadBefore, loadAfter])

In [7]:
df_sorted.sort_values(by=['deliveryTime'], inplace=True)
df_sorted = df_sorted.reset_index()
df_sorted=df_sorted.drop(columns=['index', 'level_0'])

In [8]:
df_able = df_sorted[df_sorted['Note'] == False]
df_unable= df_sorted[df_sorted['Note']]

def getDriverWorkPeriod(hours, maxCargo):
    dict_driver = []
    df_able['Assign'] = 'Wait'
    assigned = 0
    driver = 0
    #& cargo < maxCargo
    while assigned < len(df_able):
        driver += 1
        name = 'Driver ' + str(driver)
        firstDelivery = df_able['deliveryTime'][assigned]
        leaveWarehouse = firstDelivery - TRAVEL_TIME
        endWork = leaveWarehouse + hours
        leavePort = endWork - TRAVEL_TIME
        cargo = 0
        for index, row in df_able.iterrows():
            if (cargo + df_able['Quantity'][index]) <= maxCargo:
                if (df_able['Assign'][index] == 'Wait') & (row['deliveryTime'] < leavePort):
                    df_able['Assign'][index] = 'Driver ' + str(driver)
                    cargo += df_able['Quantity'][index] 
                    assigned +=1
        dict = {'Name':name,
                'First Deliver':firstDelivery,
                'Leave Warehouse':leaveWarehouse,
                'End Work':endWork,
                'Leave Port':leavePort,
                'Cargo':cargo
               }
        dict_driver.append(dict)
    df_driver=pd.DataFrame(dict_driver)
    return df_driver
df_driver = getDriverWorkPeriod(DRIVER_HOURS,MAX_CARGO)
df_driver

KeyboardInterrupt: 

In [9]:
df_unable

,Vessel Name,ETB,ETU,QC Seq Time From,QC Seq Time To,timeBefore,timeAfter,Quantity,Type,Note,deliveryTime
1,SEAMAX BRIDG,2019-07-31 14:30:00,2019-08-01 01:30:00,2019-07-31 16:30:00,2019-08-01 00:30:00,02:00:00,01:00:00,2,Berthing,True,2019-07-31 13:00:36
6,LAS VEGAS,2019-07-31 13:30:00,2019-08-01 05:00:00,2019-07-31 14:00:00,2019-08-01 03:00:00,00:30:00,02:00:00,2,Berthing,True,2019-08-02 02:00:00
7,HONGKONG E,2019-07-31 23:00:00,2019-08-02 03:00:00,2019-08-01 00:00:00,2019-08-02 01:30:00,01:00:00,01:30:00,2,Berthing,True,2019-08-02 04:00:00


In [ ]:
priority = df_sorted[(df_sorted['timeBefore'].apply(lambda o:o.seconds/60 <= 90) & df_sorted['timeAfter'].apply(lambda o:o.seconds/60 <= 90)) | df_sorted['Note']]
def printDetails():
    print('Please take note of the following Vessel which may not have enough time to deliver:')
    for index, row in priority.iterrows():
        print(row['Vessel Name'])
    print('Please leave at: ' + (df_able['ETB'][0]+timedelta(hours=-2)).strftime("%d/%m/%Y, %H:%M:%S"))
    print('Deliver to the following sequence:')
    for index, row in df_able.iterrows():
        print(str(index+1) + '. '+ row['Assign']+ ' ' + row['Vessel Name'] + ' ' + row['deliveryTime'].strftime("%d/%m/%Y, %H:%M:%S"))

printDetails()

In [ ]:
data=[]
for index, row in df_able.iterrows():
    Berth = dict(Task=row['Vessel Name'], Start=row['ETB'], Finish=row['ETU'], Resource='Berth')
    data.append(Berth)
    QC = dict(Task=row['Vessel Name'], Start=row['QC Seq Time From'], Finish=row['QC Seq Time To'], Resource='QC')
    data.append(QC)
    Delivery = dict(Task=row['Vessel Name'], Start=row['deliveryTime'], Finish=(row['deliveryTime'] + timedelta(minutes=5)), Resource='Delivery')
    data.append(Delivery)
for index, row in df_driver.iterrows():
    Driver = dict(Task=row['Name'], Start=row['Leave Warehouse'], Finish=row['End Work'], Resource='Driver')
    data.append(Driver)

colors = {'Berth': 'rgb(171, 171, 171)',
          'QC': 'rgb(50, 129, 168)',
          'Delivery': 'rgb(255, 255, 255)',
          'Driver': 'rgb(252, 186, 3)'
         }

fig = ff.create_gantt(data, group_tasks=True, colors=colors, index_col='Resource', show_colorbar=True)
fig.show()